#                              **Project**
##                      **CH5150: Optimization Techniques I**


## Installing Dependencies

In [1]:
!pip install deap

## Importing Libraries

In [2]:
# Basic Libraries
import numpy as np
import pandas as pd
import timeit
import random

# For solving ODE & Deterministic Optimization
from scipy.integrate import solve_ivp
from scipy.optimize import minimize

# For GA
from deap import base, creator, tools, algorithms

$$\require{mhchem}$$
**1. For the parallel reactions happening in an isothermal batch reactor as described below, solve the rate equations (find 𝑘1 and 𝑘2). Use different optimization algorithms(methods) with suitable minimization solver and report the observations (Use the data given to solve for 𝑘1 and 𝑘2. In the data.xlsx, 1st column is time span, and 2 to 4 columns are 𝐶𝐴, 𝐶𝐵 and 𝐶𝐶). Solve the odes using an ode solver.**

<p align=center>$\ce{A ->[k1] B}$

<p align=center>$\ce{A ->[k2] C}$

Rate equations:

<p align=center>$\frac{dC_{A}}{dt} = -(k_{1}C_{A} + k_{2}C_{A})$

<p align=center>$\frac{dC_{B}}{dt} = k_{1}C_{A}$

<p align=center>$\frac{dC_{C}}{dt} = k_{2}C_{A}$

The choice of optimization methods for solving for the rate constants, k1 and k2, in the parallel reactions happening in an isothermal batch reactor can depend on various factors, including the nature of the problem and the characteristics of the data. The optimization techniques chosen to solve are listed below:


*   Nelder-Mead Method
*   Powell Method
*   Conjugate Gradient Method
*   BFGS Method (Broyden-Fletcher-Goldfarb-Shanno)
*   Newton Method
*   COBYLA Method (Constrained Optimization BY Linear Approximations)
*   SLSQP Method (Sequential Least SQuares Programming)

**a. Load the experimental data from the "data.xlsx" file, which contains time, CA, CB, and CC values.**


In [3]:
# Load experimental data
data = pd.read_excel("data.xlsx")
time = data.iloc[:, 0].values
C_A_exp = data.iloc[:, 1].values
C_B_exp = data.iloc[:, 2].values
C_C_exp = data.iloc[:, 3].values

**b. Define the rate equations for the parallel reactions and the ODE system to be solved.**


In [4]:
# Define the rate equations
def rate_equations(t, C, k1, k2):
    C_A, C_B, C_C = C
    dC_A_dt = -(k1 + k2) * C_A
    dC_B_dt = k1 * C_A
    dC_C_dt = k2 * C_A
    return [dC_A_dt, dC_B_dt, dC_C_dt]

**c. Implement the objective function, which calculates the sum of squared residuals between the model predictions and the experimental data.**


In [5]:
# Define the objective function to minimize using RMSE
def objective(params):
    k1, k2 = params
    C0 = [C_A_exp[0], 0, 0]  # Initial concentrations

    # Bounds for k1 and k2 only when using Nelder-Mead
    if method == 'Nelder-Mead' and (k1 <= 1e-3 or k2 <= 1e-3):
        return np.inf  # Return a large value to indicate infeasibility

    # Solve the ODEs using solve_ivp
    sol = solve_ivp(rate_equations, (time[0], time[-1]), C0, args=(k1, k2), t_eval=time, method='LSODA')

    # Extract the concentrations at the specified times
    C_A_model = sol.y[0]
    C_B_model = sol.y[1]
    C_C_model = sol.y[2]

    # Calculate RMSE using residuals for C_B and C_C
    residuals = (C_B_exp - C_B_model) ** 2 + (C_C_exp - C_C_model) ** 2
    rmse = np.sqrt(np.mean(residuals))
    return rmse

*   Since CA is expected to decrease during the reaction, it is not typically included in the residuals for the optimization. Instead, the optimization aims to find the values of k1 and k2 that best fit the experimental data for the products (CB and CC).

*   Including residuals for CA could introduce redundancy in the optimization, as the concentration of CA is directly related to the concentrations of CB and CC through the rate equations. Therefore, the focus is primarily on fitting the concentrations of the products, CB and CC, to the experimental data, and the concentrations of reactants, such as CA, are implicitly accounted for in the model.

**d. Choose an optimization algorithm and set up the minimization problem to find k1 and k2 by minimizing the objective function.**


In [6]:
# Initial guesses for k1 and k2
initial_guess = [0.1, 0.1]

# List of optimization methods to compare
methods = ["Nelder-Mead", "Powell", "CG", "BFGS", "TNC", "COBYLA", "SLSQP"]

# Perform optimization and measure execution time
results = []
convergence_data = {}  # To store convergence data for plotting

for method in methods:
    setup = f"from __main__ import objective, initial_guess, solve_ivp, minimize; method='{method}'"
    timer = timeit.timeit("result = minimize(objective, initial_guess, method=method)", setup, number=1, globals=globals())
    result = minimize(objective, initial_guess, method=method)

    k1_opt, k2_opt = result.x
    rmse = result.fun  # The minimized value of the objective function (now RMSE)

    # Check if the optimization has converged
    if result.success:
        convergence_status = "Converged"
    else:
        convergence_status = "Did not converge"

    results.append({
        "Method": method,
        "Convergence": convergence_status,  # Add convergence status
        "k1_opt": k1_opt,
        "k2_opt": k2_opt,
        "Accuracy": rmse,
        "Execution Time (s)": timer
    })

    # Store convergence data
    convergence_data[method] = result.message  # Here, we store the convergence message


**e. Confirming the convergence, and comparing the accuracy, efficiency, performance and sensitivity of each optimization method.**


In [7]:
# Print and compare the results
for result in results:
    print("Method:", result["Method"])
    print("Convergence:", result["Convergence"])
    print("k1_opt:", result["k1_opt"])
    print("k2_opt:", result["k2_opt"])
    print("Accuracy:", result["Accuracy"])
    print("Execution Time (s):", result["Execution Time (s)"])
    print()

Method: Nelder-Mead

Convergence: Converged

k1_opt: 1.9998748692788715

k2_opt: 3.9997615838621496

Accuracy: 1.9624601446178055e-05

Execution Time (s): 0.14974240199808264



Method: Powell

Convergence: Converged

k1_opt: 1.9999018222782379

k2_opt: 3.999807785445786

Accuracy: 1.9544391873294202e-05

Execution Time (s): 0.30413511199731147



Method: CG

Convergence: Did not converge

k1_opt: 1.9999017781607618

k2_opt: 3.9998076688535855

Accuracy: 1.9544392917766487e-05

Execution Time (s): 0.846163782000076



Method: BFGS

Convergence: Did not converge

k1_opt: 1.9999018263820614

k2_opt: 3.9998077521324666

Accuracy: 1.9544392621755784e-05

Execution Time (s): 0.50827285899868



Method: TNC

Convergence: Did not converge

k1_opt: 1.9857536272294474

k2_opt: 3.9316821455453663

Accuracy: 0.003748239956234264

Execution Time (s): 0.7758327859992278



Method: COBYLA

Convergence: Converged

k1_opt: 1.9994226343894554

k2_opt: 3.9990020736425738

Accuracy: 3.6840527203946385e-0

*   From our comparisons we found Newton Method to be bit unreliable as it did not converge but it did come close.
*   Powell Method producedd the closest results with the least error although all error were so small and in the same order.
*   SLSQP showed the best performance beating Powell method by 0.05 seconds while producing the same result.
*   Using ODE89 in MATLAB we got the exact results for k1 = 2; k2 = 4. So we can safely conclude they are 2 & 4.

2.  **Solve Himmelblau function using suitable deterministic optimization algorithm and GA. Draw comparisons.**

\begin{equation}
\min _{x,y} (x_{1}^2 + x_{2}-11)^2 + (x_{1} + x_{2}^2-7)^2
\end{equation}

\begin{equation}
s.t. x_{1}^2 + x_{2}^2 \geq 25
\end{equation}

\begin{equation}
-5 \leq x_{1}, x_{2} \leq 5
\end{equation}

In [306]:
#Define penalty function to stay within feasible region
def himmelblau_with_penalty(x):
    x1, x2 = x
    penalty = max(0, 25 - (x1**2 + x2**2)) * 50
    return (x1**2 + x2 - 11)**2 + (x1 + x2**2 - 7)**2 + penalty

In [327]:
# Define initial guess and bounds
x0 = [-3, -3]
bounds = [(-5, 5), (-5, 5)]

### **Deterministic Optimization Algorithms**

**a. Gradient Descent**

In [328]:
result_gd = minimize(himmelblau_with_penalty, x0, method='BFGS')
print("Gradient Descent Result:", result_gd.x, "with Minimum Value:", result_gd.fun)

Gradient Descent Result: [-3.77931026 -3.283186  ] with Minimum Value: 1.4142791614207676e-15


**b. Nelder Mead Method**

In [329]:
result_nm = minimize(himmelblau_with_penalty, x0, method='Nelder-Mead')
print("Nelder-Mead Result:", result_nm.x, "with Minimum Value:", result_nm.fun)

Nelder-Mead Result: [-3.77929277 -3.2832138 ] with Minimum Value: 6.563218571685403e-08


**c. CG Method**

In [330]:
result_cg = minimize(himmelblau_with_penalty, x0, method='CG')
print("Conjugate Gradient Result:", result_cg.x, "with Minimum Value:", result_cg.fun)

Conjugate Gradient Result: [-3.77931029 -3.28318601] with Minimum Value: 5.833980405874394e-14


### **Genetic Algorithm**

In [331]:
# Define the DEAP problem as a minimization
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, -5, 5)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=2)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Register functions for evaluation, crossover, mutation, and selection
toolbox.register("evaluate", himmelblau)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

In [332]:
# Define penalty function for Genetic Algorithm
def evaluate_with_penalty(individual):
    penalty = max(0, 25 - (individual[0]**2 + individual[1]**2)) * 50
    return himmelblau(individual) + penalty,

toolbox.register("evaluate", evaluate_with_penalty)

In [333]:
# GA results
def get_best_result(population):
    best_ind = tools.selBest(population, k=1)[0]
    return best_ind, best_ind.fitness.values[0]

**a. eaSimple**

In [334]:
population = toolbox.population(n=100)
result_simple = algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=50, verbose=False)
best_simple, best_simple_fitness = get_best_result(population)
print("GA eaSimple Result:", best_simple, "with Minimum Value:", best_simple_fitness)

GA eaSimple Result: [-3.779310253414729, -3.2831859914210866] with Minimum Value: 7.416028657579182e-19


**b. eaMuPlusLambda**

In [335]:
mu, lambda_ = 50, 100
population = toolbox.population(n=mu)
result_mu_plus_lambda = algorithms.eaMuPlusLambda(population, toolbox, mu, lambda_, cxpb=0.7, mutpb=0.2, ngen=50, verbose=False)
best_mu_plus_lambda, best_mu_plus_lambda_fitness = get_best_result(population)
print("GA eaMuPlusLambda Result:", best_mu_plus_lambda, "with Minimum Value:", best_mu_plus_lambda_fitness)

GA eaMuPlusLambda Result: [-3.779310253489113, -3.2831859912216483] with Minimum Value: 1.1076340901563927e-18


**c. eaMuCommaLambda**

In [336]:
result_mu_comma_lambda = algorithms.eaMuCommaLambda(population, toolbox, mu, lambda_, cxpb=0.7, mutpb=0.2, ngen=50, verbose=False)
best_mu_comma_lambda, best_mu_comma_lambda_fitness = get_best_result(population)
print("GA eaMuCommaLambda Result:", best_mu_comma_lambda, "with Minimum Value:", best_mu_comma_lambda_fitness)

GA eaMuCommaLambda Result: [-3.779310253377747, -3.283185991286169] with Minimum Value: 7.888609052210118e-31


Gradient methods required the knowledge from Genetic Algorithm to set an initial guess to achieve the right minimization. The guess was initialised at [0,0] and it kept converging to roughly [+3.78, +3.20] which was not the right solution since the solution is the negative of that.